# 层和块

我们先回顾一下多层感知机

In [4]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.1227,  0.1640, -0.0455, -0.0950,  0.3020, -0.0244, -0.0512,  0.1067,
          0.1067, -0.0868],
        [-0.1326,  0.1018, -0.0606, -0.1750,  0.0819, -0.0393, -0.0736,  0.0962,
          0.1031, -0.0489]], grad_fn=<AddmmBackward0>)

`nn.Sequential`定义了一种特殊的`Module`

自定义块

In [5]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

实例化多层感知机的层，然后在每次调用前向传播函数时调用这些层

In [7]:
net = MLP()
net(X)

tensor([[ 0.0407, -0.0282, -0.0326,  0.0835,  0.1040,  0.0814,  0.1245, -0.1216,
         -0.0210,  0.1326],
        [-0.0736,  0.0826, -0.1168,  0.0990, -0.0369,  0.0726,  0.1905,  0.0348,
          0.0418,  0.1875]], grad_fn=<AddmmBackward0>)

顺序块

In [9]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.1613, -0.1483, -0.1588, -0.2726, -0.0395, -0.1019,  0.0494,  0.0210,
          0.1104,  0.0825],
        [ 0.2106,  0.0177, -0.0371, -0.1317, -0.0810, -0.1036,  0.0342,  0.0607,
          0.1612,  0.0227]], grad_fn=<AddmmBackward0>)

在前向传播函数中执行代码

In [7]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(-0.1102, grad_fn=<SumBackward0>)

混合搭配各种组合块的方法

In [8]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.1478, grad_fn=<SumBackward0>)